In [ ]:
#!pip install mlxtend
#%matplotlib notebook

## Import all the relevant libraries

In [ ]:
import pandas as pd
import numpy as np
import pickle
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats
import seaborn as sns
from pylab import rcParams
from sklearn import metrics
from sklearn.model_selection import train_test_split

# tensorflow
import tensorflow as tf
from tensorflow.keras import Sequential

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D,InputLayer,MaxPool2D
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

# confusion matrix
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from scipy.signal import argrelextrema


In [ ]:
from lib.normalize import acc_scalar2, frequency, magnitude
from lib.plot import plot_activity
from lib.transform import transform


In [216]:
COLUMN = "Az"
df = transform()[[COLUMN]]
transform()[['Ax','Ay','Az']].iloc[0:1000].plot(figsize=(24, 12))
# df.Time = df.Time.dt.float

hz = 100
min = int(0.5*hz)
max = 5*hz
increment = 100


def crosscorr(datax, datay, lag=0):
    return datax.corr(datay.shift(lag), method="pearson")

max = 1000
rs =pd.DataFrame(columns=['cor','lag'])

for x in range(0,int(max),5):
  offset= x
  pa = df.iloc[34:97]
  x = pa.reset_index()[COLUMN]
  y = df.iloc[offset:97-34 +offset].reset_index()[COLUMN]
  pa[COLUMN].plot(c='b',style='-.')
  c = crosscorr(x,y)
  rs= rs.append({'cor':c,'lag':offset},ignore_index=True)
n = 2
rs['max'] = rs.iloc[argrelextrema(rs['cor'].values, np.greater_equal, order=n)[0]]['cor']

rs =rs[['max','lag','cor']].dropna()
rs=rs.set_index(rs['lag'])
plt.scatter(rs.index, rs['max'], c='r')
[plt.axvline(m, c='r', linewidth=0.3) for m in rs['lag'].to_list()]
plt.plot(rs.index, rs['cor'])
plt.show()
rs.count()


In [ ]:
df = transform()

from scipy.signal import butter, lfilter, freqz


def butter_lowpass(cutoff, fs, order=5):
    return butter(order, cutoff, fs=fs, btype='low', analog=False)

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y




# Filter requirements.
order = 6
fs = 50.0       # sample rate, Hz
cutoff = 1.667  # desired cutoff frequency of the filter, Hz

# Get the filter coefficients so we can check its frequency response.
b, a = butter_lowpass(cutoff, fs, order)
ax = "y"
i=1000
data = df["A{}".format(ax)][:i].to_list()

y = butter_lowpass_filter(data, cutoff, fs, order)
df["A{}l".format(ax)] = pd.DataFrame(y)

plt.plot(df.Time[:i], data, 'b-', label='data')
plt.plot(df.Time[:i], y, 'g-', linewidth=2, label='filtered data')
plt.xlabel('Time [sec]')
plt.legend()
plt.figure(figsize=(16,12))
plt.show()


window = 10

# df['Axs'] = (df['Ax'].apply(lambda x: (x+4.0)/8.0)).rolling(window).mean()
# df['Ays'] = (df['Ay'].apply(lambda x: (x+4.0)/8.0)).rolling(window).mean()
# df['Azs'] = (df['Az'].apply(lambda x: (x+4.0)/8.0)).rolling(window).mean()
# df['Gxs'] = (df['Gx'].apply(lambda x: (x+500.0)/1000.0))
# df['Gys'] = (df['Gy'].apply(lambda x: (x+500.0)/1000.0))
# df['Gzs'] = (df['Gz'].apply(lambda x: (x+500.0)/1000.0))
# df['Axd'] = df['Axs'].diff().rolling(window).mean()
# df['Ayd'] = df['Ays'].diff().rolling(window).mean()
df["A{}d".format(ax)] = df["A{}l".format(ax)].diff().rolling(window).mean()

def plot_activity_normalized(activity, df, axis, i=1000):
    data = df[df['Activity'] == activity][["A{}".format(axis),'A{}d'.format(axis), 'A{}l'.format(axis)]][:i]
    axis = data.plot(subplots=True, figsize=(16, 12),
                     title=activity)
    for ax in axis:
        ax.legend(loc='lower left', bbox_to_anchor=(1.0, 0.5))

plot_activity_normalized('Swing Left',df,ax,i=1000)
#plt.magnitude_spectrum(df[df['Activity']=='Swing Left']['Ay'][45:1845].values, Fs=50)
plt.show()
# df


In [ ]:
plt.specgram(df[df['Activity']=='Swing Left']['Ays'][45:1845].values, Fs=50)
plt.show()

## Inspect the number of datapoints for each indivudal exercise 

In [ ]:
df['Activity'].value_counts().plot(kind='bar', title='Plotting records by activity type', figsize=(10, 4),align='center')

# Extract features
Normalize the accelerometer and gyroscope values and extract other features

In [ ]:
acc_scalar2(df)
magnitude(df)
frequency(df)

# Inspect the accelerometer and gyro data
Now it is time to review the actual contents of the sensor data for different excersizes and see if there are any issues with the data.



In [ ]:
from lib.plot import plot_datasets
plot_datasets(df,i=1000)

In [ ]:
from lib.plot import plot_datasets_magnitude

plot_datasets_magnitude(df, i=7000)


In [ ]:
from lib.plot import plot_datasets_normalized
plot_datasets_normalized(df,i=2000)

In [ ]:
from lib.plot import plot_activity_normalized
plot_activity_normalized('Swing Right',df,i=9000)

In [ ]:
plot_activity_normalized('Press',df,i=6000)

# Samping frequency 
The frequency is different for some of the sampling data due to the different methods that have been used for data recording.

Original prototype used to publish data over HTTP and esp32 was able to achieve a sampling rate of 50hz. 

After the first model has been trained the prediction data has been added to the samples, this decreased the sampling rate down to 25hz. This has been improved in the 3 version of the hardware where sampling rate including the prediction output is 100hz.

In [ ]:
df['Hz'].value_counts().sort_index().plot(kind='bar', title='Plotting records by frequency in Hz', figsize=(16, 6),align='center')

## Frequency on log scale
TODO: Need to look into why there are some negative values, seems strange.

In [ ]:
plt1 = df['Hz'].value_counts().sort_index().plot(kind='bar', title='Samples count by frequency in Hz - Log scale', figsize=(16, 6),align='center')
plt1.yaxis.grid()
plt1.set_yscale('log')

# Time Series Analysis

In [ ]:
plot_activity_normalized('Press',df,i=8000)

In [ ]:
df[50:55]
plot_activity('Swing Both Hands',df,i=1845)

In [ ]:
plt.magnitude_spectrum(df[df['Activity']=='Swing Left']['Az'][45:1845].values, Fs=50)
plt.show()

In [ ]:
plt.specgram(df[df['Activity']=='Swing Both Hands']['Am'][45:845].values,NFFT=10, Fs=50,noverlap=4,)
plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

def plot3d_e(name, sensor, norm,n):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    xs = df[df['Activity']==name][sensor+'y'+norm].head(n).values
    #ys = df[df['Activity']==name][sensor+'y'+norm].head(n).values
    ys = df[df['Activity']==name]['Time'].head(n).values
    zs = df[df['Activity']==name][sensor+'z'+norm].head(n).values
    ax.plot(xs=xs,ys=ys,zs=zs,zdir='z')
    
    #ax.contour(X=xs,Y=ys,Z=zs)
plot3d_e('Press Left','A','',1000)

In [ ]:
plot_activity_normalized('Swing Both Hands',df,i=200)

In [ ]:
label = LabelEncoder()
df['Label'] = label.fit_transform(df['Activity'])
df.head()

In [ ]:
label.classes_

In [ ]:
def get_samples2(data):
    GESTURES = label.classes_
    SAMPLES_PER_GESTURE = 40
    NUM_GESTURES = len(GESTURES)
    ONE_HOT_ENCODED_GESTURES = np.eye(NUM_GESTURES)

    inputs = []
    outputs = []

    # read each csv file and push an input and output
    for gesture_index in range(NUM_GESTURES):
      gesture = GESTURES[gesture_index]
      print(f"Processing index {gesture_index} for gesture '{gesture}'.")

      output = ONE_HOT_ENCODED_GESTURES[gesture_index]

      df = data[data['Activity'] == gesture].head(6000).copy()

      # calculate the number of gesture recordings in the file
      num_recordings = int(df.shape[0] / SAMPLES_PER_GESTURE)

      print(f"\tThere are {num_recordings} recordings of the {gesture} gesture.")

      for i in range(num_recordings):
        tensor = []
        for j in range(SAMPLES_PER_GESTURE):            
            
          index = i * SAMPLES_PER_GESTURE + j
          # normalize the input data, between 0 to 1:
          # - acceleration is between: -4 to +4
          # - gyroscope is between: -2000 to +2000
          tensor += [
          #(df['Ax'][index] + 4.0) / 8.0,
          #(df['Ay'][index] + 4.0) / 8.0,
          #(df['Az'][index] + 4.0) / 8.0,
          #(df['Gx'][index] + 500.0) / 1000.0,
          #(df['Gy'][index]  + 500.0) / 1000.0,
          #(df['Gz'][index]  + 500.0) / 1000.0,
          
              
           math.sqrt(df['Ax'][index] ** 2) / 4,
           math.sqrt(df['Ay'][index] ** 2) / 4,
           math.sqrt(df['Az'][index] ** 2) / 4,
           math.sqrt(df['Gx'][index] ** 2) / 500,
           math.sqrt(df['Gy'][index] ** 2) / 500,
           math.sqrt(df['Gz'][index] ** 2) / 500,
          #    df['Am'][index],
          #    df['Gm'][index],
          #    math.atan2(df['Ay'][index],df['Az'][index]),
        # math.atan2(-df['Ax'][index], math.sqrt(df['Ay'][index]**2 +(df['Az'][index] **2)))
          ]

        inputs.append(tensor)
        outputs.append(output)

    # convert the list to numpy array
    inputs = np.array(inputs)
    outputs = np.array(outputs)
    return inputs, outputs
inputs,outputs = get_samples2(df)

In [ ]:
# Randomize the order of the inputs, so they can be evenly distributed for training, testing, and validation
# https://stackoverflow.com/a/37710486/2020087
num_inputs = len(inputs)
randomize = np.arange(num_inputs)
np.random.shuffle(randomize)

# Swap the consecutive indexes (0, 1, 2, etc) with the randomized indexes
inputs = inputs[randomize]
outputs = outputs[randomize]

# Split the recordings (group of samples) into three sets: training, testing and validation
TRAIN_SPLIT = int(0.6 * num_inputs)
TEST_SPLIT = int(0.2 * num_inputs + TRAIN_SPLIT)

inputs_train, inputs_test, inputs_validate = np.split(inputs, [TRAIN_SPLIT, TEST_SPLIT])
outputs_train, outputs_test, outputs_validate = np.split(outputs, [TRAIN_SPLIT, TEST_SPLIT])

In [ ]:
model = Sequential()
#model.add(InputLayer(input_shape=inputs_train[0].shape))

#model.add(Conv2D(50,[2,2], activation='relu',input_shape=inputs_train[0].shape))
#model.add(Dropout(0.1))

#model.add(Conv2D(12,[2,2], activation='relu'))
#model.add(Dropout(0.2))

#model.add(Flatten())

#model.add(Dense(64, activation='relu'),input_shape=inputs_train[0].shape)
#model.add(Dropout(0.5))
NUM_GESTURES= len(label.classes_)
model = Sequential()
#model.add(Conv2D(50,[2,2], activation='relu',input_shape=inputs_train[0].shape))
model.add(Dense(16, activation='relu', input_shape =inputs_train[0].shape)) # relu is used for performance
model.add(Dense(16, activation='relu')) # relu is used for performance
model.add(Dropout(0.1)) # relu is used for performance
model.add(Dense(20, activation='relu')) # relu is used for performance
model.add(Dropout(0.2)) # relu is used for performance
model.add(Dense(32, activation='relu')) # relu is used for performance
model.add(Dense(16, activation='relu')) # relu is used for performance
model.add(Dense(NUM_GESTURES, activation='softmax')) # softmax is used, because we only expect one gesture to occur per input

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy','mae'])

In [ ]:
history = model.fit(inputs_train, outputs_train, epochs=50, batch_size=10, validation_data=(inputs_validate, outputs_validate))

In [ ]:
y_pred = model.predict_classes(inputs_test)


rounded_labels=np.argmax(outputs_test, axis=1)
rounded_labels[1]

mat = confusion_matrix(rounded_labels,y_pred)
plot_confusion_matrix(mat,class_names=label.classes_, show_normed=True,figsize=(10,10))
plt.show()

In [ ]:
# increase the size of the graphs. The default size is (6,4).
plt.rcParams["figure.figsize"] = (20,10)

# graph the loss, the model above is configure to use "mean squared error" as the loss function
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'g.', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

print(plt.rcParams["figure.figsize"])

In [ ]:
# graph the loss again skipping a bit of the start
SKIP = 0
plt.plot(epochs[SKIP:], loss[SKIP:], 'g.', label='Training loss')
plt.plot(epochs[SKIP:], val_loss[SKIP:], 'b.', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# graph of mean absolute error
mae = history.history['mae']
val_mae = history.history['val_mae']
plt.plot(epochs[SKIP:], mae[SKIP:], 'g.', label='Training MAE')
plt.plot(epochs[SKIP:], val_mae[SKIP:], 'b.', label='Validation MAE')
plt.title('Training and validation mean absolute error')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()

In [ ]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
#converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()

# Save the model to disk
open("kettle.tflite", "wb").write(tflite_model)
  
import os
basic_model_size = os.path.getsize("kettle.tflite")
print("Model is %d bytes" % basic_model_size)

In [ ]:
echo "const unsigned char model[] = {" > model.h
cat kettle.tflite | xxd -i             >> model.h
echo "};"                              >> model.h
